In [ ]:
# Required if used in google Colab
!pip install flair

     |████████████████████████████████| 322 kB 5.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 48 kB 4.5 MB/s 
     |████████████████████████████████| 788 kB 37.6 MB/s 
     |████████████████████████████████| 61 kB 459 kB/s 
     |████████████████████████████████| 19.7 MB 7.2 MB/s 
     |████████████████████████████████| 981 kB 37.0 MB/s 
     |████████████████████████████████| 3.3 MB 31.1 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 1.2 MB 36.0 MB/s 
     |████████████████████████████████| 62 kB 805 kB/s 
     |████████████████████████████████| 596 kB 47.8 MB/s 
     |████████████████████████████████| 3.3 MB 36.3 MB/s 
     |████████████████████████████████| 895 kB 37.8 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=966ed66e1ae18ed6b9738b2343d85fbd4e2977224cb43e

In [ ]:
# Required if used in google Colab and files stored in google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'ner'}

data_folder = 'resources/taggers/lv-ner-flair-edit'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

print(corpus)

2021-12-14 22:17:19,272 Reading data from /content/drive/MyDrive/lv-ner-flair-edit
2021-12-14 22:17:19,273 Train: /content/drive/MyDrive/lv-ner-flair-edit/train.txt
2021-12-14 22:17:19,283 Dev: /content/drive/MyDrive/lv-ner-flair-edit/dev.txt
2021-12-14 22:17:19,284 Test: /content/drive/MyDrive/lv-ner-flair-edit/test.txt
Corpus: 6873 train + 2242 dev + 2310 test sentences


In [ ]:
#Embeddings

#these are deprecated, but still work
#from flair.embeddings import BertEmbeddings

# recomennded up to date alternative to using BertEmbeddings
from flair.embeddings import TransformerWordEmbeddings

from flair.models import SequenceTagger

from flair.trainers import ModelTrainer

In [ ]:

label_type = 'ner'

In [ ]:
# Label dictionary creation. Datasets have been cleared from all other labels beforehand
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2021-12-14 22:17:40,928 Computing label dictionary. Progress:


100%|██████████| 6873/6873 [00:00<00:00, 11611.53it/s]

2021-12-14 22:17:41,597 Corpus contains the labels: ner (#112978)
2021-12-14 22:17:41,601 Created (for label 'ner') Dictionary with 4 tags: <unk>, O, B-person, I-person
Dictionary with 4 tags: <unk>, O, B-person, I-person


In [ ]:
# Initializing the embeddings, which triggers the download from Huggingface

embeddings = TransformerWordEmbeddings(model='FFZG-cleopatra/bert-emoji-latvian-twitter', 
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       model_max_length=576
                                       )

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

In [ ]:
# Initializing the Tagger to be trained

tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

In [ ]:
# Model trainer initialization 
trainer = ModelTrainer(tagger, corpus)

In [ ]:
# Fine tuning. Parameters selected according to Flair documentation recommendations.
# checkpoint enabled

trainer.fine_tune(data_folder,
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  checkpoint=True,
                  write_weights=True,
                  mini_batch_chunk_size=4,
                  )


2021-12-14 22:19:35,960 ----------------------------------------------------------------------------------------------------
2021-12-14 22:19:35,968 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, 

{'dev_loss_history': [tensor(0.0187, device='cuda:0'),
  tensor(0.0138, device='cuda:0'),
  tensor(0.0141, device='cuda:0'),
  tensor(0.0142, device='cuda:0'),
  tensor(0.0139, device='cuda:0'),
  tensor(0.0145, device='cuda:0'),
  tensor(0.0146, device='cuda:0'),
  tensor(0.0146, device='cuda:0'),
  tensor(0.0148, device='cuda:0'),
  tensor(0.0149, device='cuda:0')],
 'dev_score_history': [0.8712241653418125,
  0.9357495881383854,
  0.9425478767693588,
  0.9457627118644069,
  0.9489795918367347,
  0.9390142021720969,
  0.9453924914675769,
  0.9531914893617021,
  0.9529411764705882,
  0.9512605042016807],
 'test_score': 0.9390557939914163,
 'train_loss_history': [0.2599629068349757,
  0.082351806254405,
  0.07535523442458815,
  0.07415776804786306,
  0.07233249365501175,
  0.07075565277462117,
  0.07029246700411478,
  0.06912414421478975,
  0.06846908077632015,
  0.06630205454114527]}